# Lecture 14

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zyang91/spatial_data_management/blob/main/notes/lecture14.ipynb)

In [ ]:
import pygis
import ee
import geemap

ee.Initialize()

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NASA SRTM
dem = ee.Image("USGS/SRTMGL1_003")
dem_vis = {
    "min": 0,
    "max": 4000,
    "palette": "terrain",
}
Map.addLayer(dem, dem_vis, "SRTM DEM")

# Add 5-year Landsat TOA composite
landsat = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

# Add US Census States
states = ee.FeatureCollection("TIGER/2018/States")
style = {"fillColor": "00000000"}
Map.addLayer(states.style(**style), {}, "US States")
Map

In [ ]:
out_dem_stats = "dem_stats.csv"
geemap.zonal_statistics(
    dem, states, out_dem_stats, statistics_type="MEAN", scale=1000, return_fc=False
)

In [ ]:
geemap.csv_to_df(out_dem_stats).sort_values("mean")

In [ ]:
# remain the data in the cloud
fc = geemap.zonal_statistics(
    dem, states, out_dem_stats, statistics_type="MEAN", scale=1000, return_fc=True
)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(fc, {}, "mean elevation")
Map

In [ ]:
fc.aggregate_stats("ALAND")

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NLCD data
dataset = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019")
landcover = dataset.select("landcover")
Map.addLayer(landcover, {}, "NLCD 2019")

# Add US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {"fillColor": "00000000"}
Map.addLayer(states.style(**style), {}, "US States")

# Add NLCD legend
Map.add_legend(title="NLCD Land Cover", builtin_legend="NLCD")
Map

In [ ]:
nlcd_stats = "nlcd_stats.csv"
geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type="SUM",
    denominator=1000000,
    decimal_places=2,
)

In [ ]:
geemap.csv_to_df(nlcd_stats).sort_values("Class_82", ascending=False).head(10)